In [1]:
import torch
a = torch.tensor([[0.,1.,2.,3.,4.],
                  [0.,2.,4.,6.,8.]]) 

### $L_p$  normalization
* <span style="font-size:1.5em">$ \upsilon = \frac{\upsilon}{max(||\upsilon||_p, \epsilon)}$<span> 

In [2]:
torch.nn.functional.normalize(a, p=1.0, dim=0, eps=1e-12)

tensor([[0.0000, 0.3333, 0.3333, 0.3333, 0.3333],
        [0.0000, 0.6667, 0.6667, 0.6667, 0.6667]])

In [3]:
torch.nn.functional.normalize(a, p=2.0, dim=1, eps=1e-12)

tensor([[0.0000, 0.1826, 0.3651, 0.5477, 0.7303],
        [0.0000, 0.1826, 0.3651, 0.5477, 0.7303]])

### z-score normalization
* mean $ \mu = \frac{1}{N} \sum_{i=1}^N x_i $
* standard deviation $ \sigma = \sqrt{\frac{1}{N} \sum_{i=1}^N(x_i-\mu)^2}$

In [4]:
a.mean(), a.std(unbiased=False)

(tensor(3.), tensor(2.4495))

### Batch Normalization

The technique consists of adding an operation in the model just before the activation function of each layer, simply zero-centering and normalizing the inputs, then scaling and shifting the result using two new parameters per layer (one for scaling, the other for shifting). In other words, this operation lets the model learn the optimal scale and mean of the inputs for each layer.<br>
<span style="font-size:1.5em">$y=\frac{x-\mu}{\sqrt{\sigma^2+\epsilon}}*\gamma+\beta $</span>, where $\gamma=weigh, \beta=bias$

In [5]:
n = torch.nn.BatchNorm1d(5, affine=True) 
n.weight, n.bias, n.eps

(Parameter containing:
 tensor([1., 1., 1., 1., 1.], requires_grad=True),
 Parameter containing:
 tensor([0., 0., 0., 0., 0.], requires_grad=True),
 1e-05)

When affine = False, $\gamma=1, \beta=0$

In [6]:
n = torch.nn.BatchNorm1d(5, affine=False)
n.weight, n.bias, n.eps

(None, None, 1e-05)

Write myBatchNorm1d

In [7]:
import copy
def myBatchNorm1d(x, eps=1e-5, alt=False):
    y = copy.deepcopy(x)
    
    if alt:  
        for i in range(x.size(1)):  
            y[:,i] = (y[:,i]-y[:,i].flatten().mean(dim=0)) / torch.sqrt(y[:,i].flatten().var(dim=0, unbiased=False)+eps)
        return y
    
    if len(x.shape) == 2:
        mean     = x.mean(dim=0)
        variance = x.var(dim=0, unbiased=False) 
        
    elif len(x.shape) == 3:  
        y        = copy.deepcopy(x)  
        size     = y.size(1)
        mean     = []
        variance = []
        for i in range(size):
            mean.    append(y[:,i].flatten().mean(dim=0                ))
            variance.append(y[:,i].flatten().var (dim=0, unbiased=False)) 
        mean     = torch.tensor(mean).    view(1,size,1)
        variance = torch.tensor(variance).view(1,size,1) 
        
    return (x -  mean) / torch.sqrt(variance + eps)  

In [8]:
x = torch.randn(5, 3)  
torch.nn.BatchNorm1d(3, affine=False)(x)  

tensor([[ 0.1442,  0.4105,  0.6274],
        [ 0.8856, -0.9661, -1.6301],
        [-1.7968, -0.9024,  0.3107],
        [-0.1961,  1.7342,  1.2441],
        [ 0.9631, -0.2762, -0.5520]])

In [9]:
myBatchNorm1d(x)

tensor([[ 0.1442,  0.4105,  0.6274],
        [ 0.8856, -0.9661, -1.6301],
        [-1.7968, -0.9024,  0.3107],
        [-0.1961,  1.7342,  1.2441],
        [ 0.9631, -0.2762, -0.5520]])

In [10]:
x = torch.randn(2, 3, 4)    
torch.nn.BatchNorm1d(3, affine=False)(x)  

tensor([[[-0.8772,  1.1360, -0.0671,  1.3436],
         [ 1.2556, -1.5223, -0.1683, -1.3121],
         [ 1.2650,  1.2685, -1.5419,  0.0619]],

        [[-0.7943,  1.0723, -0.3127, -1.5005],
         [ 1.3729, -0.1166,  0.6561, -0.1653],
         [-0.5979, -0.5692, -0.8709,  0.9845]]])

In [11]:
 myBatchNorm1d(x)

tensor([[[-0.8772,  1.1360, -0.0671,  1.3436],
         [ 1.2556, -1.5223, -0.1683, -1.3121],
         [ 1.2650,  1.2685, -1.5419,  0.0619]],

        [[-0.7943,  1.0723, -0.3127, -1.5005],
         [ 1.3729, -0.1166,  0.6561, -0.1653],
         [-0.5979, -0.5692, -0.8709,  0.9845]]])

In [12]:
 myBatchNorm1d(x, alt=True)

tensor([[[-0.8772,  1.1360, -0.0671,  1.3436],
         [ 1.2556, -1.5223, -0.1683, -1.3121],
         [ 1.2650,  1.2685, -1.5419,  0.0619]],

        [[-0.7943,  1.0723, -0.3127, -1.5005],
         [ 1.3729, -0.1166,  0.6561, -0.1653],
         [-0.5979, -0.5692, -0.8709,  0.9845]]])